MOUNT THE DRIVE AND LOAD THE DATASET

In [1]:
#save test set predictions to a file
import os
import sys
import PIL.Image as Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
all_pairs_yang_solli.ipynb   paletteScoresKuler_solli.csv
analyse_harmonies.ipynb      paletteScoresKulerSolli.csv
comparison_yang_solli.ipynb  paletteScoresKuler_yang.csv
CycleGAN/                    paletteScoresKulerYang.csv
databaseOut2/                regression_yang_solli.ipynb
databaseOut2.zip             scoresKuler.csv
harmonies_data_2.csv         scores.png
harmonies_database/          solli_allpairs.png
harmonies_data.csv           Untitled0.ipynb
notes_harmonie.gdoc          yang_allpairs.png


In [3]:
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/harmonie'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

/content/gdrive/My Drive/Colab Notebooks/harmonie


'/content/gdrive/My Drive/Colab Notebooks/harmonie'

#open the files and get a histogram for both

In [32]:
#normalize scoresSolli to be on same range as the article scores for comparison's sake
def minmax_normalize(scoresSolli,scoresTrue):
  mi = np.min(scoresTrue)
  ma = np.max(scoresTrue)
  miSol = np.min(scoresSolli)
  maSol = np.max(scoresSolli)
  scoresSolli = (scoresSolli - miSol)/(maSol-miSol)#minmax to 0,1
  scoresSolli = (scoresSolli *(ma-mi))+mi#minmax to scoresTrue's scale
  return scoresSolli

In [35]:
scoresYang = np.loadtxt("paletteScoresKuler_yang.csv")
scoresSolli = np.loadtxt("paletteScoresKuler_solli.csv")
scoresTrue = np.loadtxt("scoresKuler.csv")
scoresSolli = minmax_normalize(scoresSolli,scoresTrue)
print("yang,solli,true : ",scoresYang.shape, scoresSolli.shape,scoresTrue.shape)

yang,solli,true :  (44986,) (44986,) (44986,)


#Does linear regression (thus a linear combination of the two methods) help ?

In [85]:
from sklearn.linear_model import LinearRegression

Data = np.vstack([scoresYang,scoresSolli])
Data = np.moveaxis(Data,(0,1),(1,0))

lin = LinearRegression()
lin = lin.fit(Data,scoresTrue)
print("les coefficients sur yang,solli sont respectivement : ",lin.coef_)
scoresCombination = lin.predict(Data)

les coefficients sur yang,solli sont respectivement :  [ 1.18288367 -0.02190748]


In [86]:
#get distance for each
MSEYang = np.mean((scoresYang-scoresTrue)**2)
MSESolli = np.mean((scoresSolli-scoresTrue)**2)
MSECombination = np.mean((scoresCombination-scoresTrue)**2)

print("erreur MSE : ")
print("yang : ",MSEYang,"solli : ",MSESolli,"combination : ",MSECombination)

erreur MSE : 
yang :  0.4062133274792358 solli :  1.0128249105828728 combination :  0.40341986430539095
